# Stable Diffusion with Panel UI

[Stable Diffusion](https://en.wikipedia.org/wiki/Stable_Diffusion#:~:text=Stable%20Diffusion%20is%20a%20deep,guided%20by%20a%20text%20prompt) is a deep learning model released in 2022. Stable Diffusion can generate detailed, realistic images from text descriptions of what the image should contain or how it should appear. 

This example demonstrates how to use [Panel](https://panel.holoviz.org) to create a web browser application for running the [Diffusers library](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb), using pre-trained models from the runwayml and CompVis repositories. See [Diffusers on github](https://github.com/huggingface/diffusers#stable-diffusion-is-fully-compatible-with-diffusers) or the blogpost on [Stable Diffusion with Diffusers](https://huggingface.co/blog/stable_diffusion) for more details on the algorithm and the training set.

## TL;DR

This app should generate images in seconds on a system with a supported GPU, or in minutes on a CPU. It has been tested for deployment on osx-M1 with its integrated GPU, linux-64 with Nvidia GPUs (Quadro RTX 8000) installed, and linux-64 with only a CPU (no GPU; much slower). 

The app downloads two models from huggingface to `~/.cache/huggingface`, which take up ~ 17GB of disk space. You can run the code as a notebook or as a deployed dashboard/app if you first install anaconda-project and then run the appropriate command for your system:

```
# run notebook on linux-64 system
anaconda-project run 

# run notebook on OSX-M1 system
anaconda-project run notebook-m1

# run panel dashboard app on linux-64 system
anaconda-project run dashboard

# run panel dashboard app on OSX-M1 system
anaconda-project run dashboard-m1
```

In [ ]:
from bokeh.resources import INLINE
from bokeh.io import output_notebook
output_notebook(resources=INLINE)

In [ ]:
import time
from contextlib import contextmanager
from collections import deque

import torch
import random
from diffusers import StableDiffusionPipeline

import panel as pn
pn.extension()

@contextmanager
def exec_time(description="Task"):
    """Context manager to measure execution time and print it to the console""" 
    st = time.perf_counter()
    yield 
    print(f"{description}: {time.perf_counter() - st:.2f} sec")

## Invoking Stable Diffusion on a prompt

The `init_model` function below will first look in the default cache location used by huggingface to find downloaded pretrained models. If these haven't been downloaded yet, it will first download the models. On subsequent restarts of the app, it will load the models from the local disk cache. 

<p>
<details><summary><u>(Optional: how to download models manually)</u></summary>
<pre>
  pipe, cache = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", return_cached_folder=True, local_files_only=False)
  pipe, cache = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",  return_cached_folder=True, local_files_only=False)
  print(cache) # to see the default cache location
</pre>
</details>

In addition to caching the pretrained model, we also initialize and cache the in-memory diffusers pipeline inside `panel.state.cache`. This ensures that each new visitor to the page does not load the same model into memory again.

The initial page load takes an extra ~10 sec or so (on a Quadro RTX 8000) and allocates the GPU memory required to load the pipeline in memory. Subsequent visitors get this pipeline from panel's cache. The memory overhead per visitor is then the amount needed to generate the image text prompt.

<details><summary><p><br><u>(Optional: performance details)</u></summary>
    
Sample output from `nvidia-smi` with memory usage information, running on a machine with Quadro RTX 8000 GPUs, after both models load:

<pre>
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:15:00.0 Off |                  Off |
| 33%   33C    P8    24W / 260W |     48MiB / 49152MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     Off  | 00000000:2D:00.0 Off |                  Off |
| 33%   40C    P8    29W / 260W |   5933MiB / 49152MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|    0   N/A  N/A      2024      G   /usr/lib/xorg/Xorg                 23MiB |
|    0   N/A  N/A      2545      G   /usr/bin/gnome-shell               20MiB |
|    1   N/A  N/A      2024      G   /usr/lib/xorg/Xorg                  4MiB |
|    1   N/A  N/A   2263594      C   .../diffusers/bin/python3.11     5925MiB |
+-----------------------------------------------------------------------------+
</pre>
</details>

In [ ]:
random_int_range = 1, int(1e6)

def init_model(model, cuda, mps, local_files_only=True):
    print(f"Init model: {model}")
    pipe = StableDiffusionPipeline.from_pretrained(model,
        torch_dtype=torch.float16 if cuda or mps else None,
        local_files_only=local_files_only)        

    # let torch choose the GPU if more than 1 is available
    if cuda:
        pipe.to(f"cuda")
    elif mps:
        pipe.to(f"mps")
        pipe.enable_attention_slicing()
    return pipe     


if 'pipelines' in pn.state.cache:
    print(f"load from cache")
    pipelines = pn.state.cache['pipelines']
    pseudo_rand_gen = pn.state.cache['pseudo_rand_gen']
else:
    cuda = torch.cuda.is_available()
    mps  = torch.backends.mps.is_available()
    device = 'cuda' if cuda else 'cpu'

    models = ['runwayml/stable-diffusion-v1-5', 
              'CompVis/stable-diffusion-v1-4']
    
    pseudo_rand_gen = torch.Generator(device=device)
    with exec_time("Load models"):
        pipelines = dict()
        for m in models:
            try:
                # try to load files from cache first
                pipelines[m] = init_model(m, cuda, mps)
            except OSError:
                pipelines[m] = init_model(m, cuda, mps, local_files_only=False)
            
    pn.state.cache['pipelines'] = pipelines
    pn.state.cache['pseudo_rand_gen'] = pseudo_rand_gen
    print(f"Save to cache")


default_model = next(iter(pipelines))

Now that we have a model, we can invoke it to generate an output (uncomment if needed):

In [ ]:
# pipelines[default_model](prompt="Chair made from twisted vines, in a manicured garden",
#                         generator=pseudo_rand_gen.manual_seed(5))[0][0]

## Cleaner interface, with parameters

That's pretty awkward to run, so let's use [Param](param.holoviz.org) to document what the user parameters are and provide a cleaner interface:

In [ ]:
import param

class StableDiffusion(param.Parameterized):
    prompt = param.String(doc="""
        Text describing the image you wish to generate""")
    
    negative_prompt = param.String(doc="""
        Text describing what _not_ to include in the image (for refining results)""")
    
    model = param.Selector(objects=list(pipelines), default=default_model, doc="""
        A pre-trained model to be used for inference""")
    
    _size_range = tuple(448 + i*2**6 for i in range(10))
    width = param.Selector(_size_range, default=_size_range[1], doc="""
        Width (in pixels) of the images to generate""")
    
    height = param.Selector(_size_range, default=_size_range[1], doc="""
        Height (in pixels) of the images to generate""")
    
    guidance_scale = param.Number(bounds=(5, 10), softbounds=(7, 8.5), step=0.1, default=7.5, doc="""
        How closely the model should try to match the prompt, at the 
        potential expense of image quality or diversity.
        Also known as CFG (Classifier-free guidance scale).""")
    
    num_steps = param.Integer(label='# of steps', bounds=(10, 75), default=30, doc="""
        How many denoising steps to take. 
        More steps takes longer but gives a more-refined image.""")

    seed = param.Integer(label='Random seed', default=1,
                         bounds=random_int_range, step=10, precedence=1, doc="""
        Seed controlling the noise values generated.""")
 
    generate = param.Event(precedence=1)
    
    param.depends("generate")
    def __call__(self, **params):
        p = param.ParamOverrides(self, params)
        pipe = pipelines[p.model]
        
        res = pipe(num_inference_steps=p.num_steps, generator=pseudo_rand_gen.manual_seed(p.seed),
                   **{k:p[k] for k in ['prompt', 'negative_prompt', 'guidance_scale', 'height', 'width']})

        return res.images[0]
    
sd = StableDiffusion()

Now that we have a Parameterized object, we can invoke it to generate an output (uncomment if needed):

In [ ]:
# sd(prompt="Chair made from twisted vines, in a manicured garden", seed=5, guidance_scale=8)

See `help(sd)` for all the options available from the Python prompt to control how this image is generated. You can try various prompts, such as:

  1. Wildflowers on a mountain side 
  2. A dream of a distant planet, with multiple moons
  3. Valley of flowers in the Himalayas
  
If the results for your prompt are not what you were hoping for, you can add hints like "yellow" to a negative prompt to remove yellow flowers from the image from prompt 1.

Users can set the heights and widths of the generated image as they like, but note that the models were trained on images with resolution of 512x512, and image quality degrades if deviating from that resolution. 

For a given prompt and set of parameters, the specific image generated is deterministic, with results controlled by a random seed. Stable diffusion starts with an initial noisy image, with the goal of removing Gaussian noise in each inference step in a way that makes it more likely that the text description would apply to this image. The seed value determines the specific noise values, determining which specific image is ultimately generated. 


## Simple Panel app

Now it's documented and ready to use from Python, but not everyone is comfortable with the command prompt, so let's make a [Panel](https://panel.holoviz.org) app to package up this functionality for anyone to use. The above class actually works already as a very simple Panel app generating and displaying an image determined by widgets for each parameter (uncomment if needed):

In [ ]:
# pn.Row(sd.param, sd.__call__)

## Full-featured Panel app

The simple app works, but let's be a bit more ambitious and add a gallery, plus saving parameters to the URL so that we can easily select our favorite outputs and store them or send them as URL links. We'll also customize some of the appearance and behavior of the default widgets.

We'll first create a little HTML-based Gallery class using Panel to hold the various images generated so far:

In [ ]:
from bokeh.models.formatters import PrintfTickFormatter
from panel.layout.base import ListLike
from panel.reactive import ReactiveHTML
from panel.viewable import Viewer, Viewable

class Gallery(ListLike, ReactiveHTML):
    """Collection of thumbnails that, when selected, restore the associated image and its parameters"""
    
    objects = param.List(item_type=Viewable)
    current = param.Integer(default=None)
    margin = param.Integer(0)

    _template = """
    <div id="gallery" style="display: grid; width: 350; height: 550; grid-template-columns: 1fr 1fr 1fr;">
    {% for img in objects %}
      <div id="img" name="{{ img.name }}" onclick=${script('click')}>${img}</div>
    {% endfor %}
    </div>
    """
    
    _scripts = {
        'click': """
          const id = event.target.parentNode.parentNode.parentNode.id;
          data.current = Number(id.split('-')[1]);
          """}

Now let's make a more full-featured Panel application using this gallery and the above Parameterized class.

When rendered with a template, the sidebar should ideally start out collapsed with only the `Prompt` text box visible. Opening the sidebar provides more options. 

By default this full-featured class randomizes the seed for each new image generated, but previously generated images can be reproduced if the seed value is specified along with the prompt and other parameters. To make it simple to return to specific images, the app URL is updated with the seed used to generate that image, so that returning to that URL will reproduce that specific image.

In [ ]:
class ModelUI(Viewer):
    model = param.Parameter(StableDiffusion())
    gallery = param.ClassSelector(class_=Gallery, default=Gallery(min_height=100), precedence=-1)
    generate_image = param.Event(precedence=1)
    
    def __init__(self, **params):
        self.history = deque(maxlen=15)
        super().__init__(**params)
        self.gallery.param.watch(self._restore_history, 'current')
        self._restore = False
        self._image_container = pn.pane.PNG(style={'border': '1px solid black'}, 
                                            height=self.model.height,
                                            width=self.model.width)
        # ensure seed always starts out being set
        self.model.seed = random.randint(*self.model.param.seed.bounds)
        # internal variable used to ignore repeat event on generate if prompt triggers callback
        self._prompt_event = False
        self._on_load()

    @contextmanager
    def _toggle(self, attr: str, value: bool):
        # toggle state of bool attribute inside context
        # if exception raised by code inside the contextmanager, set state back to original and rethrow
        init_state = getattr(self, attr)
        try:
            setattr(self, attr, value)
            yield
            setattr(self, attr, not(value))
        except Exception as ex:
            setattr(self, attr, init_state)
            raise ex

    def _update_query_params(self):
        """
        Remove all params first since update_query will only update the non-default values. 
        If the current URL has non-default values, those will be incorrect unless it is first cleared
        """
        pn.state.location.search = ''
        pn.state.location.update_query(**self._url_params)

    def _update_image_container(self, image):
        """update the object and the container size"""
        self._image_container.object = image
        self._image_container.height = self.model.height
        self._image_container.width  = self.model.width


    def _restore_history(self, event):
        """
        Load image from cache and update URL to reflect parameters used to generate image.
        Also update the seed in the end similar so generating another image does not
        recreate the restored image from history.
        """
        if event.new is None:
            return
        self.gallery.current = None
        state, image = self.history[event.new]
        # discard_events will not allow widgets to update
        with self._toggle('_restore', value=True):
            self.model.param.update(state)
        self._update_image_container(image)
        self._update_query_params()
        # Also update the seed so `generate_image` doesn't recreate same image
        self.model.seed = random.randint(*self.model.param.seed.bounds)
 
    @property
    def _state(self):
        return {k: v for k, v in self.model.param.values().items() if k != 'name'}
    
    @property
    def _url_params(self):
        # only capture state that deviates from default
        state = {key: getattr(self.model, key) for key, val in self.model.param.defaults().items()
                 if key != 'name' and getattr(self.model, key) != val}
        return state
    
    def _on_load(self):
        if pn.state.location and pn.state.location.query_params:
            self.model.param.update(pn.state.location.query_params)
            self.param.trigger('generate_image')

    @param.depends('model.prompt', 'generate_image', watch=True)
    def image(self):
        if self._restore or not self.model.prompt:
            return

        # user entered prompt, then hit generate; callback invoked on 'prompt';
        # now event triggered from generate
        if self._prompt_event and self.generate_image:
            self._prompt_event = False
            return
        self._prompt_event = True if not self.generate_image else False

        with exec_time(f"Generate {self.model.prompt}"):
            image = self.model()
            image_seed = self.model.seed

        if len(self.gallery) == self.history.maxlen:
            # Oldest element from history will be dropped
            self.gallery.remove(self.gallery[0])
        
        self.gallery.append(pn.pane.PNG(image.resize((100, 100))))
        # store full state in history
        self.history.append((self._state, image))

        self._update_query_params()
        # update seed at the end
        self.model.seed = random.randint(*self.model.param.seed.bounds)
        self._update_image_container(image)

    def _sidebar_widgets(self):
        return pn.Param(self.model.param, widgets = {
                'height': pn.widgets.DiscreteSlider,
                'width' : pn.widgets.DiscreteSlider,
                'guidance_scale': {'formatter': PrintfTickFormatter(format='%.1f')},
                'seed': pn.widgets.IntInput,
                'prompt': {'visible': False},
                'negative_prompt': {'visible': False},
                'generate': {'visible': False}})

    def _main_panel(self):
        return pn.Column(pn.Row(pn.Column(self.model.param.prompt, self.model.param.negative_prompt, 
                                          sizing_mode='stretch_width'),
                                pn.Param(self.param.generate_image, 
                                         widgets={'generate_image': {'button_type': 'success', 
                                                                     'height': 110, 'width': 30}})),
                         pn.Row(pn.panel(self._image_container, loading_indicator=True), self.gallery))
    
    def __panel__(self):
        return pn.Row(
            pn.Column(self._sidebar_widgets()),
            pn.Column(self._main_panel(), sizing_mode='stretch_width'))

    
sdui = ModelUI(name='Stable Diffusion with Panel UI')

sdui

The above app should work well in a notebook cell, but when we serve this as a standalone web page, it's nice to embed it in a full-page template (not shown here in the notebook for formatting reasons):

In [ ]:
logo_pn = """<a href="http://panel.pyviz.org">
    <img src="https://panel.pyviz.org/_static/logo_stacked.png" 
    width=108 height=91 align="left" margin=10px>"""

logo_diffusers = """<a href="https://huggingface.co/docs/diffusers/index">
    <img src="./thumbnails/diffusers_logo.png" 
    width=198 height=102 align="left" margin=10px>"""

desc = """
    The <a href="http://panel.pyviz.org">Panel</a> library from 
    <a href="https://holoviz.org/">HoloViz</a> 
    lets you make widget-controlled apps. This Panel app lets you use the
    <a href="https://huggingface.co/docs/diffusers/index">diffusers</a> library to
    generate images from pretrained diffusion models."""

template = pn.template.MaterialTemplate(title=sdui.name)

template.sidebar.append(pn.Column(pn.Row(logo_diffusers, logo_pn),
                                  pn.panel(desc, width=300, margin=(20, 5)),
                                  sdui._sidebar_widgets()))

template.main.append(pn.Column(sdui._main_panel(), sizing_mode='stretch_width'))

template.servable();

Now you can launch and share this app with `panel serve stable_diffusion.ipynb` or `anaconda-project run dashboard` or `anaconda-project run dashboard-m1` !